In [8]:
import torch
import torch.optim as optim
import torch.utils.data as data
from torchsummary import summary
from tqdm import tqdm
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import Dataset,DataLoader

import os
import pickle
import numpy as np
import matplotlib.pyplot as plt


In [9]:
from torchvision.datasets import ImageNet

In [10]:
def index_classes(items):
    idx = {}
    for i in items:
        if (i not in idx):
            idx[i] = len(idx)
    return idx

class MiniImagenet(data.Dataset):
    """
    [[Source]](https://github.com/learnables/learn2learn/blob/master/learn2learn/vision/datasets/mini_imagenet.py)
    **Description**
    The *mini*-ImageNet dataset was originally introduced by Vinyals et al., 2016.
    It consists of 60'000 colour images of sizes 84x84 pixels.
    The dataset is divided in 3 splits of 64 training, 16 validation, and 20 testing classes each containing 600 examples.
    The classes are sampled from the ImageNet dataset, and we use the splits from Ravi & Larochelle, 2017.
    **References**
    1. Vinyals et al. 2016. “Matching Networks for One Shot Learning.” NeurIPS.
    2. Ravi and Larochelle. 2017. “Optimization as a Model for Few-Shot Learning.” ICLR.
    **Arguments**
    * **root** (str) - Path to download the data.
    * **mode** (str, *optional*, default='train') - Which split to use.
        Must be 'train', 'validation', or 'test'.
    * **transform** (Transform, *optional*, default=None) - Input pre-processing.
    * **target_transform** (Transform, *optional*, default=None) - Target pre-processing.
    * **download** (bool, *optional*, default=False) - Download the dataset if it's not available.
    **Example**
    ~~~python
    ~~~
    """
    def __init__(
        self,
        root,
        mode='train',
        transform=None,
        target_transform=None):
        super(MiniImagenet, self).__init__()
        self.root = os.path.expanduser(root)
        if not os.path.exists(self.root):
            os.mkdir(self.root)
        self.transform = transform
        self.target_transform = target_transform
        self.mode = mode
        pickle_file = os.path.join(self.root, 'mini-imagenet-cache-' + mode + '.pkl')

        with open(pickle_file, 'rb') as f:
            self.data = pickle.load(f)

   #     self.x = torch.from_numpy(self.data["image_data"]).permute(0, 3, 1, 2).float()
        self.x =self.data["image_data"]
        self.y = np.ones(len(self.x))

        # TODO Remove index_classes from here
        self.class_idx = index_classes(self.data['class_dict'].keys())
        for class_name, idxs in self.data['class_dict'].items():
            for idx in idxs:
                self.y[idx] = self.class_idx[class_name]

    def __getitem__(self, idx):
        data = self.x[idx]
        if self.transform:
            data = self.transform(data)
        return data, self.y[idx]

    def __len__(self):
        return len(self.x)


In [11]:
import math
import torch.nn as nn
from kernel import CannyFilter

In [14]:
transform=T.Compose([T.ToTensor(),T.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
train_set=MiniImagenet(root="D:/Dataset/Mini-ImageNet",mode='train',transform=transform)
torch.manual_seed(0)
train_set, val_set = torch.utils.data.random_split(train_set, [30000, 8400])


In [15]:
img=train_set[321][0]*0.5+0.5

In [ ]:
plt.imshow(T.ToPILImage()(img))

In [1]:
import random 

In [2]:
wandb

NameError: name 'wandb' is not defined

In [ ]:
for _ in range(5):
    img_idx=random.randint(0,len(train_set)-1)
    canny_kernel=CannyFilter(k_gaussian=3,k_sobel=3,blur=False)
    blur_kernel=CannyFilter(k_gaussian=1,k_sobel=3,blur=False)
    img=train_set[img_idx][0]*0.5+0.5
    outputs=canny_kernel(img.unsqueeze(0))
    fig,ax=plt.subplots(ncols=4,figsize=(8,16))
    noblur_outputs=blur_kernel(img.unsqueeze(0))

    ax[0].imshow(T.ToPILImage()(img))
    ax[1].imshow(T.ToPILImage()(outputs[0][0]))
    ax[2].imshow(T.ToPILImage()(outputs[1][0]),cmap='gray')
    ax[3].imshow(T.ToPILImage()(noblur_outputs[1][0]),cmap='gray')




In [9]:
train_dl = torch.utils.data.DataLoader(dataset=train_set,
                                          batch_size=84,
                                          shuffle=True,
                                          drop_last=True)
valid_dl = torch.utils.data.DataLoader(dataset=val_set,
                                          batch_size=84,
                                          shuffle=False,
                                          drop_last=True)

In [10]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(3, 32,3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

                
            
        self.layer4= torch.nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        
        self.layer5 = torch.nn.Sequential(
            nn.Conv2d(64, 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
                    
        self.classifier = torch.nn.Sequential(
            nn.Linear(5 * 5 * 64, 128, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(128, 64, bias=True))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out=self.layer4(out)
      #  out = self.layer5(out)
     #   print(out.shape)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.classifier(out)
        return out

In [11]:
import torchvision.models as models
model = models.resnet18(num_classes=64)
torch.manual_seed(0)
config_=dict()
config_["lr"]=0.005
config_["crit"]=nn.CrossEntropyLoss()
config_["optim"]=optim.Adam(model.parameters(),lr=config_["lr"])
config_["save_dir"]="models/"
config_["epochs"]=30
config_['run_name']=f'edge_finetune_{stop_epoch}'
config_["accuracy"]=True
config_["log_interval"]=100
config_["device"]="cuda"

NameError: name 'stop_epoch' is not defined

In [12]:
def train(config,train_dl,valid_dl,model,kernel=None,stop_epoch=10):
    '''
    config.optimizer: Training optimizer eg) Adam,SGD....
    config.criterion: Loss eg) CrossEntropy, 
    config.lr: learning_rate
    train_dl,valid_dl: Dataloader
    model: target model
    save_path
    '''
    canny_kernel=CannyFilter(k_gaussian=1,k_sobel=3,blur=False,use_cuda=True)
    model.to(config["device"])
    wandb.init(project="Augmentation_Strategy")
    wandb.run.name = config['run_name']
    wandb.run.save()
    canny_kernel.to(config["device"])
    max_val_accuracy=0
    for epoch in tqdm(range(config["epochs"])):
            print(f'Epochs : {epoch+1}')
            model.train()
            avg_train_loss=0
            correct=0
            total=0
            for batch_idx,(img,label) in enumerate(train_dl):
                #Zero_grad_Optimizer
                config["optim"].zero_grad()
                img=img.to(config["device"])
                if epoch<stop_epoch:#
                    with torch.no_grad():
                       _,img=canny_kernel(img)
                       img=img.expand(-1,3,-1,-1)
              #  img=kernel(img)-img
                label=label.to(config["device"]).long()
                output=model(img)

                #Measure Loss
                loss=config["crit"](output,label)
                loss.backward()

                if config["accuracy"]:
                    _, predicted = torch.max(output.data, 1)
                    total+=label.size()[0]
                    correct += (predicted == label).sum().item()

                #Update Parameters
                config["optim"].step()
                avg_train_loss+=loss.item()
                if batch_idx % config["log_interval"]==0:
                    wandb.log({
                        "Train Accuracy": correct/(total),
                        "Train Loss": avg_train_loss/(batch_idx+1)})
            model.eval()
            valid_loss=0
            correct=0
            total=0

            for batch_idx,(img,label) in enumerate(valid_dl):

                img=img.to(config["device"])
                label=label.to(config["device"]).long()

                output=model(img)
                #Measure Loss
                loss=config["crit"](output,label)

                valid_loss+=loss.item()

                if config["accuracy"]:
                    _, predicted = torch.max(output.data, 1)
                    total+=label.size()[0]
                    correct += (predicted == label).sum().item()
            avg_valid_loss=valid_loss/(len(valid_dl))
            wandb.log({
                "epoch": epoch,
                "Val Accuracy": correct/total,
                "Val Loss": avg_valid_loss})
            if (correct/total)>max_val_accuracy:
                max_val_accuracy=correct/total
                print('Validation Result is better, saving the new model')
                torch.save(model.state_dict(), config["save_dir"]+config_['run_name'])

    return config,model

In [13]:
import wandb
model = models.resnet18(num_classes=64)
torch.manual_seed(0)
for stop_epoch in range(1,30):
    config_=dict()
    config_["lr"]=0.005
    config_["crit"]=nn.CrossEntropyLoss()
    config_["optim"]=optim.Adam(model.parameters(),lr=config_["lr"])
    config_["save_dir"]="models/"
    config_["epochs"]=30
    config_['run_name']=f'edge_finetune_{stop_epoch}'
    config_["accuracy"]=True
    config_["log_interval"]=100
    config_["device"]="cuda"
    config_,model=train(config_,train_dl,valid_dl,model,stop_epoch=stop_epoch)


wandb: Currently logged in as: hslrock (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
C:\Users\admin\anaconda3\lib\site-packages\torch\tensor.py:746: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  warnings.warn("The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad "
  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

Epochs : 1


  3%|██▊                                                                                | 1/30 [00:59<28:50, 59.66s/it]

Validation Result is better, saving the new model
Epochs : 2


...

Epochs : 29


 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [27:55<00:57, 57.62s/it]

Epochs : 30


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [28:42<00:00, 57.40s/it]

Validation Result is better, saving the new model


In [68]:
config_["device"]

'cuda'